In [ ]:
import numpy as np
import pandas as pd
from scipy.special import logit
import lightgbm as lgb

In [ ]:
train = pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
test = pd.read_csv("../input/santander-customer-transaction-prediction/test.csv")
train = train.drop('ID_code', axis = 1)
train.head()

In [ ]:
test_id = test.ID_code
test = test.drop('ID_code', axis = 1)
test.head()

## Feature Engineering & Model Building (refer to Dott)
[922 in 3 minutes](https://www.kaggle.com/dott1718/922-in-3-minutes/comments)

#### Since the variables are independent, use each of them combined with the frequency of each of its value as predictor to predict the probability of purchasing with LGBM, and then calculate the logit value of each probability and sum up all the logit values (by 200 predictors) to get the final result.

In [ ]:
features = [x for x in train.columns if x.startswith("var")]

hist_df = pd.DataFrame()
for var in features:
    var_stats = train[var].append(test[var]).value_counts()
    hist_df[var] = pd.Series(test[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1

ind = hist_df.sum(axis = 1) != 200
var_stats = {var: train[var].append(test[ind][var]).value_counts() for var in features}

pred = 0
for var in features:
    model = lgb.LGBMClassifier(**{'learning_rate': 0.05, 
                                  'max_bin': 165, 
                                  'max_depth': 5, 
                                  'min_child_samples': 150,
                                  'min_child_weight': 0.1, 
                                  'min_split_gain': 0.0018, 
                                  'n_estimators': 41,
                                  'num_leaves': 6, 
                                  'reg_alpha': 2.0, 
                                  'reg_lambda': 2.54, 
                                  'objective': 'binary', 
                                  'n_jobs': -1})
    model = model.fit(np.hstack([train[var].values.reshape(-1, 1),
                      train[var].map(var_stats[var]).values.reshape(-1, 1)]), train["target"].values)
    pred += logit(model.predict_proba(np.hstack([test[var].values.reshape(-1, 1),
                  test[var].map(var_stats[var]).values.reshape(-1, 1)]))[:, 1])
    
pd.DataFrame({"ID_code": test_id, "target": pred}).to_csv("submission.csv", index = False)